In [125]:
import pandas as pd
import string

## Load and clean data from Rayyan

In [126]:
df = pd.read_csv("data/articles.csv")
df.head()

,key,title,year,month,day,journal,issn,volume,issue,pages,...,url,language,publisher,location,abstract,notes,doi,keywords,pubmed_id,pmc_id
0,rayyan-354359269,An experience of automated assessment in a lar...,2021.0,1,1,NaN,NaN,29.0,5.0,NaN,...,NaN,NaN,NaN,NaN,The 2020 pandemic imposed new demands on teach...,"RAYYAN-LABELS: 2021,CAE,vpl,mctest,moodle,pyth...",NaN,NaN,NaN,NaN
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,2021.0,1,1,NaN,NaN,29.0,5.0,NaN,...,NaN,NaN,NaN,NaN,Continuous assessment is essential in educatio...,"RAYYAN-LABELS: 2021,CAE,java,javassess,test ca...",NaN,NaN,NaN,NaN
2,rayyan-354359271,A tool for evaluating computer programs from s...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Computer science studies are more and more pop...,"RAYYAN-LABELS: 2021,EASEAI,pylint,python,metri...",NaN,Software,NaN,NaN
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,This study proposes an automated programming a...,"RAYYAN-LABELS: 2021,ICKII,test cases,correctne...",NaN,NaN,NaN,NaN
4,rayyan-354359273,Analysis of an automatic grading system within...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Reliable and pedagogically sound automated fee...,"RAYYAN-LABELS: 2021,CEP,vpl,java,mule,test cas...",NaN,NaN,NaN,NaN


### Clean data

In [127]:
df.dropna(axis=1, inplace=True)
df.drop(columns={'month', 'day'}, inplace=True)
df.head()

,key,title,authors,abstract,notes
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"RAYYAN-LABELS: 2021,CAE,vpl,mctest,moodle,pyth..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"RAYYAN-LABELS: 2021,CAE,java,javassess,test ca..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"RAYYAN-LABELS: 2021,EASEAI,pylint,python,metri..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"RAYYAN-LABELS: 2021,ICKII,test cases,correctne..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"RAYYAN-LABELS: 2021,CEP,vpl,java,mule,test cas..."


In [128]:
df.notes = df.notes.map(lambda input_str: input_str.lstrip('RAYYAN-LABELS: '))

df['screener_1'] = df.notes.map(lambda labels: [label for label in labels.split(',') if ':' not in label])
df['screener_2'] = df.notes.map(lambda labels: [label for label in labels.split(',') if ':' in label])

df.head()

,key,title,authors,abstract,notes,screener_1,screener_2
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,cor...","[2021, CAE, vpl, mctest, moodle, python, corre...","[tool:vpl, comparison:with-without]"
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[2021, CAE, java, javassess, test cases, corre...","[skill:correctness, technique:dsl-rules, feedb..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,cod...","[2021, EASEAI, pylint, python, metrics, gamifi...","[skill:readability, skill:maintainability, tec..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,test cases,correctness,semi-automat...","[2021, ICKII, test cases, correctness, semi-au...","[skill:correctness, evaluation:auto-grading, t..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[2021, CEP, vpl, java, mule, test cases, corre...","[skill:correctness, technique:dsl-rules, techn..."


#### Prepend screener 1's labels with topics

In [129]:
def convert_labels(labels):
    output = []
    for label in labels:
        if label in ['vpl', 'mctest', 'moodle', 'javassess', 'pylint', 'coderunner', 'checkstyle', 'sarfgen', 'appgrader', 'better code hub', 'matlab grader',
                     'stepik', 'earsketch', 'pmd', 'pyassessment', 'mule', 'uncode', 'jdbc', 'pass', 'cpplint', 'web-cat', 'cppcheck', 'bluej', 'submitty',
                     'juypter', 'webta', 'docker', 'pyta', 'progedu', 'siette', '2tsw']:
            label = 'tool:' + label
        elif label in ['java', 'python', 'opengl', 'C', 'unknown language', 'sql', 'glsl', 'c++', 'android', 'javascript', 'c#', 'lex', 'yacc',
                       'language  agnostic', 'vb', 'scala', 'matlab', 'ocaml', 'ruby', 'scheme', 'web languages', 'assesmbly', 'typescript', 'prolog']:
            label = 'language:' + label
        elif label in ['correctness', 'code quality', 'oop', 'maintainability', 'graphics', 'complexity', 'documentation', 'concurrency',
                       'teaching context free grammar', 'best practices', 'refactoring', 'distrbuted', 'code design', 'test suite quality', 'productivity']:
            label = 'skill:' + label
        elif label in ['experience report', 'demo paper', 'evaluation paper']:
            label = 'type:' + label
        elif label in ['pre-defined questions', 'test cases', 'pattern matching', 'model solution req', 'dsl for exercise gen', 'mutants',
                       'code repair for feedback', 'metrics', 'program repair', 'static analysis', 'control flow graph', 'model-based', 'unit test expansion',
                       'dynamic analysis', 'ml', 'error prediction', 'fault localisation', 'ci', 'cluster', 'data-driven', 'matching rules', 'language stuctures', 'ci/cd', 'test case generation',
                       'token pattern matching', 'knowledge base', 'reflection', 'hipos', 'graphical output', 'nlp', 'dsl', 'antipatterns', 'mutation testing', 'uml', 'program synthesis']:
            label = 'technique:' + label
        elif label in ['internal exams', 'internal assignments', 'microsoft-dev204.1x', 'internal tutorial', 'leetcode', 'code defenders dataset', 'edX',
                       'github dataset', 'bespoke assignment', 'hackerrank', 'codechef', 'grok', 'dataset available']:
            label = 'data:' + label
        elif label in ['semi-automatic', 'oj']:
            label = 'approach:' + label
        elif label in ['personalised feedback', 'gamified', 'pre-defined feedback', 'plagiarism', 'hints', 'nudge theory', 'visual feedback', 'manual personalised feedback']:
            label = 'feedback:' + label
        elif label in ['compared to human', 'compared to other tools']:
            label = 'comparison:' + label

        output.append(label)

    return output

Check for outstanding labels to classify

In [130]:
df.screener_1 = df.screener_1.apply(lambda labels: convert_labels(labels))

temp_df = df.screener_1.map(lambda labels: [label for label in labels if ':' not in label and not label.isdigit() and not label[0].isupper()])
temp_df[temp_df.str.len() > 0]

20                          [lms, adaptive learning]
39                                        [accuracy]
48     [item response theory, classical test theory]
56                                [lms, mood design]
62                                       [networked]
64                         [alogthmic understanding]
67                                        [template]
79                              [group contribution]
85                                          [to tag]
86                                          [to tag]
90                                          [to tag]
95                                         [atelier]
99                              [heavily structured]
100                        [operating system grader]
101                                         [to tag]
108                                 [error recovery]
113                                         [to tag]
118                                         [to tag]
Name: screener_1, dtype: object

In [131]:
df.head()

,key,title,authors,abstract,notes,screener_1,screener_2
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,cor...","[2021, CAE, tool:vpl, tool:mctest, tool:moodle...","[tool:vpl, comparison:with-without]"
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[2021, CAE, language:java, tool:javassess, tec...","[skill:correctness, technique:dsl-rules, feedb..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,cod...","[2021, EASEAI, tool:pylint, language:python, t...","[skill:readability, skill:maintainability, tec..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,test cases,correctness,semi-automat...","[2021, ICKII, technique:test cases, skill:corr...","[skill:correctness, evaluation:auto-grading, t..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[2021, CEP, tool:vpl, language:java, tool:mule...","[skill:correctness, technique:dsl-rules, techn..."


### Separate out year published and publication location

In [132]:
# Get year from list adapted from: https://stackoverflow.com/questions/363944/python-idiom-to-return-first-item-or-none

df['year_published'] = df.screener_1.apply(lambda labels: next(iter([label for label in labels if label.isdigit()] or []), None))
df.screener_1 = df.screener_1.apply(lambda  labels: [label for label in labels if not label.isdigit()])

df.screener_1 = df.screener_1.map(lambda labels: [label for label in labels if ":" in label])
df.screener_2 = df.screener_2.map(lambda labels: [label for label in labels if ":" in label])

df.head()

,key,title,authors,abstract,notes,screener_1,screener_2,year_published
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,cor...","[tool:vpl, tool:mctest, tool:moodle, language:...","[tool:vpl, comparison:with-without]",2021
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[language:java, tool:javassess, technique:test...","[skill:correctness, technique:dsl-rules, feedb...",2021
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,cod...","[tool:pylint, language:python, technique:metri...","[skill:readability, skill:maintainability, tec...",2021
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,test cases,correctness,semi-automat...","[technique:test cases, skill:correctness, appr...","[skill:correctness, evaluation:auto-grading, t...",2021
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[tool:vpl, language:java, tool:mule, technique...","[skill:correctness, technique:dsl-rules, techn...",2021


#### Clean tags

In [133]:
punct = string.punctuation.replace(':', '').replace('#', '') + " "

In [134]:
def clean_tags(labels):
    return [label.translate(str.maketrans(dict.fromkeys(punct, "_"))) for label in labels]

In [135]:
df.screener_1 = df.screener_1.apply(lambda labels: clean_tags(labels))
df.screener_2 = df.screener_2.apply(lambda labels: clean_tags(labels))

df.head()

,key,title,authors,abstract,notes,screener_1,screener_2,year_published
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,cor...","[tool:vpl, tool:mctest, tool:moodle, language:...","[tool:vpl, comparison:with_without]",2021
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[language:java, tool:javassess, technique:test...","[skill:correctness, technique:dsl_rules, feedb...",2021
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,cod...","[tool:pylint, language:python, technique:metri...","[skill:readability, skill:maintainability, tec...",2021
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,test cases,correctness,semi-automat...","[technique:test_cases, skill:correctness, appr...","[skill:correctness, evaluation:auto_grading, t...",2021
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[tool:vpl, language:java, tool:mule, technique...","[skill:correctness, technique:dsl_rules, techn...",2021


In [136]:
df['tags'] = df.screener_1 + df.screener_2
df.tags = df.tags.apply(lambda tags: list(sorted(set(tags))))

df = df.drop(columns=['screener_1', 'screener_2', 'notes'])

df.head()

,key,title,authors,abstract,year_published,tags
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,2021,"[comparison:with_without, language:python, ski..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,"[approach:semi_automatic, data:internal_exams,..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,2021,"[feedback:gamified, feedback:personalised_feed..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,2021,"[approach:semi_automatic, comparison:compared_..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,2021,"[data:internal_assignments, evaluation:auto_gr..."


## Validate data

In [137]:
required_prefixes = {'approach', 'language', 'type', 'skill', 'technique', 'data', 'evaluation', 'comparison', 'feedback', 'tool'}

In [140]:
def validate(labels):
    paper_prefixes = []
    for label in labels:
        paper_prefixes.append(label.split(':')[0])

    return [prefix for prefix in required_prefixes if prefix not in paper_prefixes]

pd.concat([df.title, df.tags.apply(lambda labels: validate(labels))], axis=1)

,title,tags
0,An experience of automated assessment in a lar...,"[data, approach, evaluation, feedback]"
1,Semiautomatic generation and assessment of Jav...,"[type, comparison]"
2,A tool for evaluating computer programs from s...,"[type, data, approach, comparison, evaluation]"
3,Virtual Teaching Assistant for Grading Program...,"[type, tool, feedback]"
4,Analysis of an automatic grading system within...,"[type, approach, comparison, feedback]"
...,...,...
118,Learning and teaching numerical methods with a...,"[type, data, approach, comparison, tool, langu..."
119,Ask-Elle: an Adaptable Programming Tutor for H...,"[type, approach, comparison, tool, language]"
120,An AI system for coaching novice programmers,"[type, approach, comparison, evaluation]"
121,Automatic extraction of AST patterns for debug...,"[type, approach, tool, evaluation]"


### Example analytics count of all tags

In [139]:
pd.Series([tag for tags in df.tags for tag in tags]).value_counts()

skill:correctness            103
technique:test_cases          71
data:internal_assignments     65
technique:unit_testing        49
language:java                 34
                            ... 
tool:better_code_hub           1
tool:appgrader                 1
skill:documentation            1
language:vb                    1
language:prolog                1
Length: 172, dtype: int64

## Export